# HigherEdAssist: Academic Program Advisor

This assistant helps prospective students understand academic programs and offers guidance on enrollment decisions.
It includes an advanced recommendation engine that uses contextual embeddings,combining program descriptions and keywords 
to suggest programs based on student's interests and skills.
The embeddings are generated using the fine-tuned model.


## Imports

In [6]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from sentence_transformers import SentenceTransformer, util

## Initialization

In [7]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print("OpenAI API Key exists and begins", openai_api_key[:8])
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


## Load the Fine-Tuned Model

In [8]:
sbert_model = SentenceTransformer('fine_tuned_academic_program_model')

## Defining basic data for programs

### Program information with descriptions and keywords.

In [9]:
program_info = {
    "computer science": {
        "description": "The Computer Science program offers a comprehensive curriculum covering programming, algorithms, data structures, software engineering, and artificial intelligence. Graduates are prepared for careers in software development, research, and technology management.",
        "prerequisites": "High school mathematics, problem-solving skills, and a passion for technology.",
        "career_outcomes": "Software Engineer, Data Scientist, Researcher, System Analyst.",
        "keywords": ["technology", "programming", "software", "algorithms", "AI", "innovation"]
    },
    "business administration": {
        "description": "The Business Administration program provides insights into management, finance, marketing, and entrepreneurship. It emphasizes practical skills through internships, case studies, and collaborative projects.",
        "prerequisites": "Strong communication skills, basic understanding of mathematics, and an interest in leadership and management.",
        "career_outcomes": "Business Manager, Marketing Specialist, Financial Analyst, Entrepreneur.",
        "keywords": ["management", "finance", "marketing", "leadership", "entrepreneurship", "business"]
    },
    # Add additional programs as needed.
}

### Precompute Program Embeddings using Contextual Information


In [ ]:
# Combine the program description and keywords for a richer context.
program_embeddings = {}
for prog, details in program_info.items():
    description = details.get("description", "")
    keywords_text = " ".join(details.get("keywords", []))
    combined_text = f"{description} {keywords_text}"
    embedding = sbert_model.encode(combined_text, convert_to_tensor=True)
    program_embeddings[prog] = embedding

## Tools

### Tool 1: Get Program Description

In [ ]:
def get_program_description(program_name):
    program = program_info.get(program_name.lower())
    if program:
        desc = (
            f"**{program_name.title()}**\n\n"
            f"**Description:** {program['description']}\n\n"
            f"**Prerequisites:** {program['prerequisites']}\n\n"
            f"**Career Outcomes:** {program['career_outcomes']}\n\n"
            "Please consult with an academic advisor for personalized advice."
        )
    else:
        desc = f"Sorry, we don't have information on the program '{program_name}'. Please try another program or contact our admissions office."
    return desc

program_description_function = {
    "name": "get_program_description",
    "description": "Provides detailed information about a specific academic program.",
    "parameters": {
        "type": "object",
        "properties": {
            "program_name": {
                "type": "string",
                "description": "The name of the academic program (e.g., Computer Science, Business Administration)"
            }
        },
        "required": ["program_name"],
        "additionalProperties": False
    }
}

### Tool 2: Get Program Recommendation Based on Interests and Skills with Contextual Embeddings

In [ ]:
def get_program_recommendation(interests, skills, weight_interests=0.6, weight_skills=0.4):
    """
    Recommend programs based on a combination of interests and skills.
    Compute separate embeddings for interests and skills, then combine them using weighted addition.
    Compare against contextual embeddings computed from the program description and keywords.
    """
    interests_embedding = sbert_model.encode(interests, convert_to_tensor=True)
    skills_embedding = sbert_model.encode(skills, convert_to_tensor=True)
    combined_embedding = weight_interests * interests_embedding + weight_skills * skills_embedding
    
    recommendations = {}
    for prog, prog_embedding in program_embeddings.items():
        cosine_sim = util.cos_sim(combined_embedding, prog_embedding)
        similarity_score = cosine_sim.item()
        if similarity_score > 0.2:
            recommendations[prog.title()] = similarity_score
    
    if recommendations:
        sorted_recs = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
        rec_text = "Based on your interests and skills, we recommend the following programs:\n\n"
        for program, score in sorted_recs:
            rec_text += f"- **{program}** (similarity: {score:.2f})\n"
        rec_text += "\nPlease consult with an academic advisor for personalized guidance."
    else:
        rec_text = "No program recommendations found based on your provided interests and skills. Please refine your input or contact admissions for further assistance."
    
    return rec_text

program_recommendation_function = {
    "name": "get_program_recommendation",
    "description": "Provides program recommendations based on user interests and skills using contextual embeddings (combining program descriptions and keywords) and cosine similarity with weighted inputs.",
    "parameters": {
        "type": "object",
        "properties": {
            "interests": {
                "type": "string",
                "description": "A comma-separated list of your academic interests (e.g., technology, management, finance)"
            },
            "skills": {
                "type": "string",
                "description": "A comma-separated list of your skills (e.g., programming, analytical thinking, communication)"
            }
        },
        "required": ["interests", "skills"],
        "additionalProperties": False
    }
}

### Combine tools into a list

In [10]:
tools = [
    {"type": "function", "function": program_description_function},
    {"type": "function", "function": program_recommendation_function},
]

## System Message

In [11]:
system_message = """You are HigherEdAssist, a friendly academic program advisor.
You help prospective students understand academic programs and offer guidance on enrollment decisions.
Feel free to ask follow-up questions or request further clarifications.
Always encourage students to consult with an academic advisor for personalized recommendations.
"""

## Chat function

In [12]:
def chat(message, history):
    # Build conversation history.
    messages = [{"role": "system", "content": system_message}]
    for h in history:
        # Check if h is a tuple or list with exactly 2 elements.
        if isinstance(h, (list, tuple)) and len(h) == 2:
            user_msg, assistant_msg = h
        elif isinstance(h, dict):  # If h is a dictionary, use keys 'user' and 'assistant'
            user_msg = h.get("user", "")
            assistant_msg = h.get("assistant", "")
        else:
            # If h does not match expected formats, skip it.
            continue
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    messages.append({"role": "user", "content": message})
    
    # Call the OpenAI API.
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    # Process any tool calls if indicated.
    tool_call_count = 0
    max_tool_calls = 3
    while response.choices[0].finish_reason == "tool_calls" and tool_call_count < max_tool_calls:
        tool_call_count += 1
        bot_message = response.choices[0].message
        messages.append({"role": "assistant", "content": bot_message.content, "tool_calls": bot_message.tool_calls})
        
        for tool_call in bot_message.tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            result = call_tool_function(function_name, arguments)
            tool_response = {
                "role": "tool",
                "content": json.dumps(result),
                "tool_call_id": tool_call.id
            }
            messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

## Helper Function to Route the Right Tool Call

In [13]:
def call_tool_function(function_name, arguments):
    if function_name == "get_program_description":
        program_name = arguments.get('program_name')
        desc = get_program_description(program_name)
        return {"program_name": program_name, "description": desc}
    
    elif function_name == "get_program_recommendation":
        interests = arguments.get('interests')
        skills = arguments.get('skills')
        recs = get_program_recommendation(interests, skills)
        return {"interests": interests, "skills": skills, "recommendations": recs}
    
    return {"error": f"Unknown function: {function_name}"}

## User Interface

In [14]:
#  Define the Gradio UI and Launch the Application
with gr.Blocks() as demo:
    gr.Markdown("# HigherEdAssist: Academic Program Advisor")
    
    chatbot = gr.ChatInterface(
        fn=chat,
        chatbot=gr.Chatbot(height=500),
        type="messages",
        title="Chat with HigherEdAssist",
        description="Ask about academic programs, get detailed program descriptions, or request recommendations based on your interests and skills.",
        examples=[
            "Tell me about Computer Science.",
            "What is Business Administration all about?",
            "Which program should I enroll in if I'm interested in technology, innovation, and programming?",
            "Can you recommend programs if I like management, finance, and leadership? My skills include strong communication and analytical thinking."
        ]
    )
    
    with gr.Accordion("Available Programs", open=False):
        program_list = ", ".join([p.title() for p in program_info.keys()])
        gr.Markdown(program_list)
    
    with gr.Accordion("How It Works", open=False):
        gr.Markdown(
            """
            - Type your query about a specific program or ask for recommendations based on your interests and skills.
            - The assistant provides detailed program descriptions including curriculum highlights, prerequisites, and career outcomes.
            - The recommendation tool uses contextual embeddings (combining program descriptions and keywords) with a weighted approach to better match your input.
            - Feel free to ask follow-up questions or request clarifications.
            - Please consult with an academic advisor for personalized guidance.
            """
        )

# Launch the application
demo.launch()

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:310: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.
